### Dependencies 

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from starfish import StructuredLLM, data_factory
from starfish.common.env_loader import load_env_file

load_env_file()

2025-04-13 23:57:50 | INFO     | starfish.common.env_loader | env_loader.py:52 | Loaded 9 environment variables from /Users/zhengisamazing/1.python_dir/starfish/.env


True

### Structured LLM - Single
#### 1. Model provider LLM Call

In [3]:
first_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York")
first_response.data

[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, New York City has an estimated population of over 8.4 million people, making it the most populous city in the United States.'}]

In [4]:
print(first_llm.render_prompt_printable(city_name="New York", num_records=5))


📝 CONSTRUCTED MESSAGES:

Role: user
Content:
Facts about city New York.


You are asked to generate exactly 5 records and please return the data in the following JSON format: 
[
    {
    "question": ""  //  (required),
    "answer": ""  //  (required)
    }
    ...
]

Required fields: question, answer


End of prompt



#### 2. Customized Openai Compatible Model provider LLM Call

In [5]:
first_llm = StructuredLLM(
    model_name="hyperbolic/deepseek-ai/DeepSeek-V3-0324",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{'name': 'question', 'type': 'str'}, 
                   {'name': 'answer', 'type': 'str'}],
    model_kwargs={"temperature": 0.7}
)

first_response = await first_llm.run(city_name="New York", num_records=5)
first_response.data

[{'question': 'What is the nickname of New York City?',
  'answer': 'The Big Apple'},
 {'question': 'Which famous statue is located in New York Harbor?',
  'answer': 'The Statue of Liberty'},
 {'question': 'What is the name of the largest park in Manhattan?',
  'answer': 'Central Park'},
 {'question': 'Which borough of New York City is known for its diverse culture and arts scene?',
  'answer': 'Brooklyn'},
 {'question': 'What is the name of the famous theater district in New York City?',
  'answer': 'Broadway'}]

#### 3. Local LLM

In [6]:
### Local model
first_llm = StructuredLLM(
    model_name="ollama/gemma3:1b",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York", num_records=5)
first_response.data

2025-04-13 23:58:00 | INFO     | starfish.core.llm.proxy.litellm_adapter | litellm_adapter.py:94 | Ensuring Ollama model gemma3:1b is ready...
2025-04-13 23:58:00 | INFO     | starfish.core.llm.backend.ollama_adapter | ollama_adapter.py:63 | Starting Ollama server...
2025-04-13 23:58:01 | INFO     | starfish.core.llm.backend.ollama_adapter | ollama_adapter.py:79 | Ollama server started successfully
2025-04-13 23:58:01 | INFO     | starfish.core.llm.backend.ollama_adapter | ollama_adapter.py:129 | Found model gemma3:1b
2025-04-13 23:58:01 | INFO     | starfish.core.llm.backend.ollama_adapter | ollama_adapter.py:232 | Model gemma3:1b is already available
2025-04-13 23:58:01 | INFO     | starfish.core.llm.proxy.litellm_adapter | litellm_adapter.py:103 | Model gemma3:1b is ready, making API call...


[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, the population of New York City is approximately 8.8 million people.'}]

In [7]:
### Clean it up
from starfish.core.llm.backend.ollama_adapter import stop_ollama_server
await stop_ollama_server()

2025-04-13 23:58:08 | INFO     | starfish.core.llm.backend.ollama_adapter | ollama_adapter.py:254 | Stopping Ollama server...
2025-04-13 23:58:09 | INFO     | starfish.core.llm.backend.ollama_adapter | ollama_adapter.py:305 | Ollama server stopped successfully


True

### Structured LLM - Workflow
#### 1. Two LLM

In [8]:
from starfish.core.llm.utils import merge_structured_outputs
from starfish import StructuredLLM 
first_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
)

first_response = await first_llm.run(city_name="New York", num_records=5)


second_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="""You will be given a list of question and answer pairs, 
please rate each individually about its accuracy, funny and conciseness. 
rating are from 1 to 10, 1 being the worst and 10 being the best. 
lets also rank them among themself so from 1 being the best.
Here is question and answer pairs: {{QnA_pairs}}""",
    output_schema=[{"name": "accuracy", "type": "int"}, {"name": "funny", "type": "int"}, {"name": "conciseness", "type": "int"}, {"name": "rank", "type": "int"}],
    model_kwargs={"temperature": 1},
)

second_response = await second_llm.run(QnA_pairs=first_response.data)

### Merge result:
merge_structured_outputs(first_response.data, second_response.data)


[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, New York City has an estimated population of over 8.4 million people.',
  'accuracy': 9,
  'funny': 2,
  'conciseness': 9,
  'rank': 2},
 {'question': 'What is the official nickname of New York City?',
  'answer': "New York City is commonly known as 'The Big Apple.'",
  'accuracy': 10,
  'funny': 3,
  'conciseness': 10,
  'rank': 1},
 {'question': 'Which river borders New York City to the west?',
  'answer': 'The Hudson River borders New York City to the west.',
  'accuracy': 9,
  'funny': 1,
  'conciseness': 9,
  'rank': 4},
 {'question': 'What is the tallest building in New York City?',
  'answer': 'As of 2023, One World Trade Center is the tallest building in New York City, standing at 1,776 feet.',
  'accuracy': 10,
  'funny': 2,
  'conciseness': 10,
  'rank': 1},
 {'question': 'What famous park is located in the heart of Manhattan?',
  'answer': 'Central Park is the famous park located in the heart 

In [9]:
print(first_llm.render_prompt_printable(city_name="New York", num_records=5))


📝 CONSTRUCTED MESSAGES:

Role: user
Content:
Facts about city New York.


You are asked to generate exactly 5 records and please return the data in the following JSON format: 
[
    {
    "question": ""  //  (required),
    "answer": ""  //  (required)
    }
    ...
]

Required fields: question, answer


End of prompt



In [10]:
print(second_llm.render_prompt_printable(QnA_pairs=first_response.data))


📝 CONSTRUCTED MESSAGES:

Role: user
Content:
You will be given a list of question and answer pairs, 
please rate each individually about its accuracy, funny and conciseness. 
rating are from 1 to 10, 1 being the worst and 10 being the best. 
lets also rank them among themself so from 1 being the best.
Here is question and answer pairs: |QnA_pairs| : [{"question": "What is the population of New York City?", "answer": "As of 2023, New York City has an estimated population of over 8.4 million people."}, {"question": "What is the official nickname of New York City?", "answer": "New York City is commonly known as 'The Big Apple.'"}, {"question": "Which river borders New York City to the west?", "answer": "The Hudson River borders New York City to the west."}, {"question": "What is the tallest building in New York City?", "answer": "As of 2023, One World Trade Center is the tallest building in New York City, standing at 1,776 feet."}, {"question": "What famous park is located in the heart o

In [11]:
@data_factory(max_concurrency=50)
async def workflow(city_name, num_records_per_city):
    print(f"Processing city: {city_name}!")
    first_llm = StructuredLLM(
        model_name="openai/gpt-4o-mini",
        prompt="Facts about city {{city_name}}.",
        output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    )

    first_response = await first_llm.run(city_name=city_name, num_records=num_records_per_city)


    second_llm = StructuredLLM(
        model_name="openai/gpt-4o-mini",
        prompt="""You will be given a question and answer pair, 
                please rate each individually about accuracy, funny and conciseness. 
                rating are from 1 to 10, 1 being the worst and 10 being the best. 
                Here is question and answer pair: {{QnA_pairs}}""",
        output_schema=[{"name": "accuracy", "type": "int"}, {"name": "funny", "type": "int"}, {"name": "conciseness", "type": "int"}],
        model_kwargs={"temperature": 1},
    )

    second_response = await second_llm.run(QnA_pairs=first_response.data)

    ### Merge result:
    final_output = merge_structured_outputs(first_response.data, second_response.data)

    return final_output


final_output = workflow.run(data = [
                                        {'city_name': 'New York'},
                                        {'city_name': 'Los Angeles'},
                                        {'city_name': 'Chicago'},
                                        {'city_name': 'Houston'},
                                        {'city_name': 'Miami'}
                                    ], num_records_per_city=5)

2025-04-13 23:58:43 | INFO     | starfish.core.data_factory.factory | factory.py:180 | 
2. Creating master job...
2025-04-13 23:58:43 | INFO     | starfish.core.data_factory.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
Processing city: New York!
Processing city: Los Angeles!
Processing city: Chicago!
Processing city: Houston!
Processing city: Miami!
2025-04-13 23:58:44 | INFO     | starfish.core.data_factory.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-13 23:58:45 | INFO     | starfish.core.data_factory.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-13 23:58:46 | INFO     | starfish.core.data_factory.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-13 23:58:47 | INFO     | starfish.core.data_factory.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-13 23:58:48 | INFO     | starfish.core.data_f

/Users/zhengisamazing/Library/Caches/pypoetry/virtualenvs/starfish-T7IInzTH-py3.11/lib/python3.11/site-packages/ric
h/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [22]:
from typing import Any, Dict

from starfish.core.structured_llm import StructuredLLM
from starfish.utils.data_factory import data_factory

# -> List[Dict] job hooks hook-up


# Add callback for error handling
def handle_error(err_str: str):
    print(f"Error occurred: {err_str}")


async def handle_record_complete(data: Any, state: Dict[str, Any]):
    print(f"Record complete: {data}")


async def handle_duplicate_record(data: Any, state: Dict[str, Any]):
    print(f"Record duplicated: {data}")


@data_factory(
    storage="local", max_concurrency=50, state={}, on_record_complete=[handle_record_complete, handle_duplicate_record], on_record_error=[handle_error]
)
async def get_city_info_wf(city_name, region_code):
    structured_llm = StructuredLLM(
        model_name="openai/gpt-4o-mini",
        prompt="Facts about city {{city_name}} in region {{region_code}}.",
        output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
        model_kwargs={"temperature": 0.7},
    )
    output = await structured_llm.run(city_name=city_name, region_code=region_code)
    return output.data


# Execute with batch processing
results = get_city_info_wf.run(
    cities=["Paris", "Tokyo", "New York", "London"],
    num_facts=3
)

results = get_city_info_wf.run(
    city_name= ["Berlin", "Rome"],
    region_code=["DE", "IT"],
)


results = get_city_info_wf.run(
    data=[{"city_name": "Berlin"}, {"city_name": "Rome"}],
    region_code=["DE", "IT"],
    city_name="Beijing",  ### Overwrite the data key
    # num_records_per_city = 3
)


TypeError: default_input_converter() missing 1 required positional argument: 'data'